In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))


from utils import process_articles_directory
from sentence_transformers import SentenceTransformer, util

/Users/alexislimozin/.pyenv/versions/3.11.9/envs/ada/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
data_path = os.path.abspath(os.path.join(os.getcwd(), 'data'))
articles_path = os.path.join(data_path, 'plaintext_articles')

result = process_articles_directory(articles_path)
df = pd.DataFrame(result, columns=['Article_Title', 'Related_Subjects', 'Description'])
print(f"There are : {len(df)} articles")
df.head()

There are : 4598 articles


,Article_Title,Related_Subjects,Description
0,Second Crusade,"[Military History and War, Religious disputes]",The Second Crusade was the second major crusad...
1,Navassa Island,[North American Geography],"Navassa Island (French: La Navase, Haitian Kre..."
2,Evan Rachel Wood,"[Actors, models and celebrities]","Evan Rachel Wood (born September 7, 1987) is a..."
3,Tropical Storm Henri (2003),[Storms],Tropical Storm Henri was a weak tropical storm...
4,Final Fantasy Adventure,[Computer & Video games],Final Fantasy Adventure (originally released i...


In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
df['Article_Title_embedding'] = df['Article_Title'].apply(model.encode, engine='numba', engine_kwargs={'parallel':True})
df['Description_embedding'] = df['Description'].apply(model.encode, engine='numba', engine_kwargs={'parallel':True})

In [17]:
for row in df.itertuples():
    score = util.dot_score(row.Article_Title_embedding,row.Description_embedding)
    if score < 0.2:
        print(f"{row.Article_Title} score: {score}")
        print(f"Description: {row.Description}")

Cheers score: tensor([[0.1901]])
Description: Cheers was an American situation comedy produced by Charles-Burrows-Charles Productions in association with Paramount Television for NBC. Cheers was created by the team of James Burrows, Glen Charles, and Les Charles. The show was set in the eponymous Cheers bar (itself named for the toast " Cheers") in Boston, Massachusetts, where a group of locals met to drink and generally have fun. The show's theme song was written and performed by Gary Portnoy with its famous refrain, "where everybody knows your name", that also became the show's tagline.
Amur score: tensor([[0.1967]])
Description: The Amur River (Russian: Амур; Simplified Chinese: 黑龙江; Traditional Chinese: 黑龍江; pinyin: Hēilóng Jiāng, or "Black Dragon River"; Mongolian: Хара-Мурэн, Khara-Muren or "Black River"; Manchu: Sahaliyan Ula, literal meaning "Black River") is Earth's eighth longest river, forming the border between the Russian Far East and Manchuria in China.


In [58]:
def compare_cosines(tup1, tup2):
    tup1_emb1 = tup1['Article_Title_embedding'].iloc[0]
    tup2_emb1 = tup2['Article_Title_embedding'].iloc[0]

    tup1_emb2 = tup1['Description_embedding'].iloc[0]
    tup2_emb2 = tup2['Description_embedding'].iloc[0]
    print(f"Cosine similarity between article names: {float(util.dot_score(tup1_emb1, tup2_emb1)):.4f}")
    print(f"Cosine similarity between descriptions: {float(util.dot_score(tup1_emb2, tup2_emb2)):.4f}")


In [ ]:
# Both are science fiction writers. Using only their names, the cosine similarity
# is low. But with more context, their cosine similarity is high. They are indeed
# bi-directionally linked as they had influence in each other's lives.
print("John Campbell and Isaac Asimov comparison")
john = df[df['Article_Title'] == "John W. Campbell"]
isaac = df[df['Article_Title'] == "Isaac Asimov"]
compare_cosines(john, isaac)

# It can also help to disambiguate: a limerick is a form of poetry, so these two
# should be linked if we only take into account the article name. But the Limerick
# article actually talks about the city in Ireland, not the form of poetry.
print("\nLimerick and Poetry comparison")
limerick = df[df['Article_Title'] == "Limerick"]
poetry = df[df['Article_Title'] == "Poetry"]
compare_cosines(limerick, poetry)

# In the case where the articles should be linked but the description isn't similar
# like winter and summer which are opposing, the description cosine similarity is
# lower, but closer to the article name similarity than in the case of disambiguation.
print("\nWinter and Summer comparison")
winter = df[df['Article_Title'] == "Winter"]
summer = df[df['Article_Title'] == "Summer"]
compare_cosines(winter, summer)

John Campbell and Isaac Asimov comparison
Cosine similarity between article names: 0.2764
Cosine similarity between descriptions: 0.5171

Limerick and Poetry comparison
Cosine similarity between article names: 0.5433
Cosine similarity between descriptions: 0.1685

Winter and Summer comparison
Cosine similarity between article names: 0.6008
Cosine similarity between descriptions: 0.4943


Intuitively, articles should be linked if the cosine similarity between descriptions is higher than a certain threshold, while being close tothe article names cosine similarity. But it is easier to let a neural network determine the weights while passing on both the article names and descriptions embeddings as variables for each node.